## COMP90042 Project 2022: Rumour Detection and Analysis on Twitter
## Task 2
### Group Member:
Yizhou Wang 873236 \
Siqi Zhou 903274 \
Haoyu Bai 956490 \

##Processing the COVID_data

In [ ]:
import csv
rumour_id = []
non_rumour_id = []
csv_file = open("covid_id_label.csv")

csvreader = csv.reader(csv_file)

header = []
header = next(csvreader)

for row in csvreader:
  if row[1] == "1":
    rumour_id.append(int(row[0]))
  else:
    non_rumour_id.append(int(row[0]))



covid_tweet = []

covid_raw = open("covid_trimmed_data.txt")

for line in covid_raw:
    line = line[:-1]
    tweet_ids = line.split(",")
    covid_tweet.append(tweet_ids)

import os
import json

def getJSONwithID(tweet_id, path):
    str_tweet_json = ""
    if (not os.path.isfile("./" + path + "/" + tweet_id + ".json")):
        return {}
    with open(path + "/" + tweet_id + ".json", "r") as file:
        if path == "tweet-objects":
            return json.loads(file.readline())
        else:
            str_tweet_json = file.readline()
    file.close()
    str_tweet_json = str_tweet_json.replace('\\"',"\"").replace('\\\\', "\\")[1:-1]
    obj = json.loads(str_tweet_json)
    return obj


## Analysis

### HashTag Analysis

In [ ]:
### HashTag analysis

rumour_hashtag_dict = {}
for idx in rumour_id:
    source_tweet = covid_tweet[idx][0]
    source_tweet_json = getJSONwithID(source_tweet, "covid_data")
    if "entities" in source_tweet_json:
        hashtags_dict = source_tweet_json["entities"]["hashtags"]
        hashtags = [hashtag_dict["text"] for hashtag_dict in hashtags_dict]
        for hashtag in hashtags:
            hashtag = hashtag.lower().replace("_", "").replace("-","")
            if hashtag not in rumour_hashtag_dict:
                rumour_hashtag_dict[hashtag] = 1
            else:
                rumour_hashtag_dict[hashtag] += 1


sorted_rumour_hashtag_dict = {k: v for k, v in sorted(rumour_hashtag_dict.items(), key=lambda item: item[1], reverse=True)}

import pandas as pd

df = pd.DataFrame(data=sorted_rumour_hashtag_dict, index = [0])
df = (df.T)

df.to_excel('rumour_source_tweet.xlsx')

non_rumour_hashtag_dict = {}
for idx in non_rumour_id:
    source_tweet = covid_tweet[idx][0]
    source_tweet_json = getJSONwithID(source_tweet, "covid_data")
    if "entities" in source_tweet_json:
        hashtags_dict = source_tweet_json["entities"]["hashtags"]
        hashtags = [hashtag_dict["text"] for hashtag_dict in hashtags_dict]
        for hashtag in hashtags:
            hashtag = hashtag.lower().replace("_", "").replace("ー", "").replace("-", "")
            if hashtag not in non_rumour_hashtag_dict:
                non_rumour_hashtag_dict[hashtag] = 1
            else:
                non_rumour_hashtag_dict[hashtag] += 1


sorted_non_rumour_hashtag_dict = {k: v for k, v in sorted(non_rumour_hashtag_dict.items(), key=lambda item: item[1], reverse=True)}



df = pd.DataFrame(data=sorted_non_rumour_hashtag_dict, index = [0])
df = (df.T)

df.to_excel('non_rumour_source_tweet.xlsx') 

import time
from datetime import datetime
def timeFormatting(strtime):

    return datetime.strptime(strtime, '%a %b %d %H:%M:%S +0000 %Y')

non_rumour_time_dict = {"Within 1 month": 0, "1 - 6 months": 0, "6  - 12 months": 0, "1 - 2 years": 0, "2 - 5 years": 0, "more than 5 years": 0}
for idx in non_rumour_id:
    source_tweet = covid_tweet[idx][0]
    source_tweet_json = getJSONwithID(source_tweet, "covid_data")
    if "user" in source_tweet_json:
        user_created = source_tweet_json["user"]["created_at"]
        formatted_user_created = timeFormatting(user_created)
        tweet_created = source_tweet_json["created_at"]
        formatted_tweet_created = timeFormatting(tweet_created)

        diff = formatted_tweet_created -formatted_user_created

        if diff.days < 30:
            non_rumour_time_dict["Within 1 month"] += 1

        elif diff.days >= 30 and diff.days < 180:
            non_rumour_time_dict["1 - 6 months"] += 1

        elif diff.days >= 180 and diff.days < 365:
            non_rumour_time_dict["6  - 12 months"] += 1

        elif diff.days >= 365 and diff.days < 365 * 2:
            non_rumour_time_dict["1 - 2 years"] += 1

        elif diff.days >= 365 * 2 and diff.days < 365 * 5:
            non_rumour_time_dict["2 - 5 years"] += 1

        else:
            non_rumour_time_dict["more than 5 years"] += 1


print(non_rumour_time_dict)



rumour_time_dict = {"Within 1 month": 0, "1 - 6 months": 0, "6  - 12 months": 0, "1 - 2 years": 0, "2 - 5 years": 0, "more than 5 years": 0}
for idx in rumour_id:
    source_tweet = covid_tweet[idx][0]
    source_tweet_json = getJSONwithID(source_tweet, "covid_data")
    if "user" in source_tweet_json:
        user_created = source_tweet_json["user"]["created_at"]
        formatted_user_created = timeFormatting(user_created)
        tweet_created = source_tweet_json["created_at"]
        formatted_tweet_created = timeFormatting(tweet_created)

        diff = formatted_tweet_created -formatted_user_created

        if diff.days < 30:
            rumour_time_dict["Within 1 month"] += 1

        elif diff.days >= 30 and diff.days < 180:
            rumour_time_dict["1 - 6 months"] += 1

        elif diff.days >= 180 and diff.days < 365:
            rumour_time_dict["6  - 12 months"] += 1

        elif diff.days >= 365 and diff.days < 365 * 2:
            rumour_time_dict["1 - 2 years"] += 1

        elif diff.days >= 365 * 2 and diff.days < 365 * 5:
            rumour_time_dict["2 - 5 years"] += 1

        else:
            rumour_time_dict["more than 5 years"] += 1


print(rumour_time_dict)

### Sentiment Analysis

In [ ]:
### Sentiment Analysis

from pysentimiento import create_analyzer
analyzer = create_analyzer(task="sentiment", lang="en")

import csv
rumour_id = []
non_rumour_id = []
csv_file = open("covid_id_label.csv")

csvreader = csv.reader(csv_file)

header = []
header = next(csvreader)

for row in csvreader:
  if row[1] == "1":
    rumour_id.append(int(row[0]))
  else:
    non_rumour_id.append(int(row[0]))

covid_feature = []

tsv_file = open("feature3_covid.tsv")

tsvreader = csv.reader(tsv_file, delimiter = "\t")

header = []
header = next(tsv_file)

for row in tsvreader:
  event = row[0]
  tweets = event.split("[SEP]")
  covid_feature.append(tweets)

total_sentiment_dict = {}
index = 0
for i in range(len(covid_feature)):
  for j in range(len(covid_feature[i])):
    if index % 1000 == 0:
      print(index)
    str_index = str(i) + "," + str(j)
    tweet = covid_feature[i][j]
    sentiment_dict = analyzer.predict(tweet).probas
    total_sentiment_dict[str_index] = sentiment_dict
    index += 1


non_rumour_sentiment_dict = {"NEU": 0, "POS": 0, "NEG": 0}
for i in non_rumour_id:
  for j in range(len(covid_feature[i])):
    if j == 1:
      continue
    else:
      str_ij = str(i) + "," + str(j)
      sentiment_dict = total_sentiment_dict[str_ij]
      max_sentiment = max(sentiment_dict, key=sentiment_dict.get)
      non_rumour_sentiment_dict[max_sentiment] += 1

print(non_rumour_sentiment_dict)


rumour_sentiment_dict = {"NEU": 0, "POS": 0, "NEG": 0}
for i in rumour_id:
  for j in range(len(covid_feature[i])):
    if j == 1:
      continue
    else:
      str_ij = str(i) + "," + str(j)
      sentiment_dict = total_sentiment_dict[str_ij]
      max_sentiment = max(sentiment_dict, key=sentiment_dict.get)
      rumour_sentiment_dict[max_sentiment] += 1
    
print(rumour_sentiment_dict)

### Topic analysis 

#### loading tweet from file

In [ ]:
from filehelper import *

rumourlist, nonrumourlist = covidreader("covid_id_label.csv", "covid_trimmed_data.txt")
print("numeber of rumoulist = {};       number of nonrumourlist =   {}".format(len(rumourlist),len(nonrumourlist)))

#### tweet preprocesssing


In [ ]:
import datetime
from datetime import date
# we want to anaylize how does the topic varies by time; have to sort the list by time first.
def createtimegetter(tweet):
    creatdate = datetime.datetime.strftime(datetime.datetime.strptime(tweet["created_at"],'%a %b %d %H:%M:%S +0000 %Y'), '%Y%m%d')
    return creatdate

#sorting
rumourlist = sorted(rumourlist, key=lambda x: createtimegetter(x))
nonrumourlist = sorted(nonrumourlist, key=lambda x: createtimegetter(x))

# getting time from string representation
def gettimefromstring(timestamp):
    return datetime.datetime(int(timestamp[0:4]),int(timestamp[4:6]) ,int(timestamp[6:]))

#seperate by 30 days periods
def SplitByMonth(sourcelist,days = 30):
    periodenddate = gettimefromstring(createtimegetter(sourcelist[0]))
    intervel = datetime.timedelta(days=days)
    periodenddate += intervel
    chunkedsourcelist = []
    temp = []
    for source in sourcelist:
        if gettimefromstring(createtimegetter(source)) < periodenddate:
            temp.append(source)
        else:
            periodenddate += intervel
            chunkedsourcelist.append(temp)
            temp = []
    return chunkedsourcelist

nonrumourtrend = SplitByMonth(nonrumourlist)
rumourtrend = SplitByMonth(rumourlist)


In [ ]:
import re
import nltk
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
tt = TweetTokenizer()
lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()
stopwords = set(stopwords.words("english")) 
def preprocessing(tweet,lemmatizer, stopword,tokenizer):
    ps = PorterStemmer()
    new_tweet = []
    for chunk in tweet.split(" "):
        if chunk.startswith('http') or chunk.startswith("@") or not (re.search('[a-zA-Z]',chunk)) :
            continue
        chunk = re.sub(r'[^0-9a-zA-Z]+', '', chunk)
    #tokenization
        chunks = tokenizer.tokenize(chunk)
        for chunk in chunks:
        #lemmentizer
            lemma = lemmatizer.lemmatize(chunk, "v")
            if lemma == chunk:
                lemma = lemmatizer.lemmatize(chunk, "n")
            if not lemma in stopword and lemma is not None:
                if len(lemma) > 4:
                    new_tweet.append(lemma.lower())
    if new_tweet != []:
        result = ps.stem(" ".join(new_tweet))
        if len(result) > 4:
            return result
    return None

rumourlist = list(filter(None,[preprocessing(tweet["text"],lemmatizer,stopwords,tt) for tweet in rumourlist]))
nonrumourlist = list(filter(None,[preprocessing(tweet["text"],lemmatizer,stopwords,tt) for tweet in nonrumourlist]))

#### hot topic analysis

In [ ]:
from collections import Counter
def mostcommonword(event):
    counter1 = Counter()
    for tweet in event:
        if tweet != None:
            counter1 += Counter(tweet.split(" "))
    return counter1
rumourdict = mostcommonword(rumourlist)
nonrumourdict = mostcommonword(nonrumourlist) 
sorted_rumour= sorted(rumourdict.items(), key=lambda kv: kv[1],reverse = True)
sorted_nonrumour= sorted(nonrumourdict.items(), key=lambda kv: kv[1],reverse = True)

# check the most common 30 words
print(sorted_rumour[:30])
print(sorted_nonrumour[:30])


#### General topic anaylysis 

In [ ]:
# lda factory
import gensim
def ldamodel_maker(wordlist):
    alllist =[]
    for word in wordlist:
        splitedword = word.split(' ')
        alllist.append(splitedword)

    dictionary = gensim.corpora.Dictionary(alllist)
    bow_corpus = [dictionary.doc2bow(doc) for doc in alllist]
    return gensim.models.LdaModel(bow_corpus, num_topics=9, id2word=dictionary)



In [ ]:
import matplotlib.pyplot as plt
# the grpah plot helper function
def modelplot(lda_model_tfidf,isrummor):
    rumortitle = "rumor" if isrummor else "nonrumor"
    fig, axs = plt.subplots(3, 3,figsize=(20, 10))
    axs = axs.flatten()
    for idx, topic in lda_model_tfidf.print_topics(-1):
        words = topic.split(" + ")
        words = [word.split("*") for word in words]
        word = [word[0] for word in words]
        score = [word[1] for word in words]
        ax = axs[idx]
        ax.barh(score,word, height=0.8)
        ax.set_title('{} tweet most common topic(tf-idf): {}'.format(rumortitle,idx))
    plt.show()


##### ploting the overall hot rumour topics 

In [ ]:
rumourlda = ldamodel_maker(rumourlist)
modelplot(rumourlda,True)

##### ploting the overall hot non - rumour topics

In [ ]:
nonruorlda = ldamodel_maker(nonrumourlist)
modelplot(nonruorlda,False)

### Trend analysis

In [ ]:
#pre processing the nonrumour by months
preprocessed_nonrumortrend = []
for sublist in nonrumourtrend:
    preprocessed_nonrumortrend.append(list(filter(None,[preprocessing(tweet["text"],lemmatizer,stopwords,tt) for tweet in sublist])))

#preprocessing the rumor by months
preprocessed_rumortrend = []
for sublist in rumourtrend:
    preprocessed_rumortrend.append(list(filter(None,[preprocessing(tweet["text"],lemmatizer,stopwords,tt) for tweet in sublist])))

#loading the lda model to a list handler
ldamodalcontainer = []
for nonrusublist, rusublist in zip(preprocessed_nonrumortrend,preprocessed_rumortrend):
    ldamodalcontainer.append((ldamodel_maker(nonrusublist),ldamodel_maker(rusublist)))

ldamodalcontainer = []
for nonrusublist, rusublist in zip(preprocessed_nonrumortrend,preprocessed_rumortrend):
    ldamodalcontainer.append((ldamodel_maker(nonrusublist),ldamodel_maker(rusublist)))

#### nonrumour topic by time

In [ ]:
import pandas as pd
import numpy as np

nonrutotal = []
nonrutime = []
nonrumon = 1
for (nonrulda,rulda) in ldamodalcontainer:
    nonru_ntopic = 1
    for idx, topic in nonrulda.print_topics(num_words=10):
        nonrutime.append("The {} mon, top {} topic".format(nonrumon,nonru_ntopic))
        words = topic.split(" + ")
        words = [word.split("*") for word in words]
        words = [word[1] for word in words]
        nonrutotal.append(words)
        nonru_ntopic += 1
        
    nonrumon = nonrumon + 1
print("=========== nonrumor topic by time ======================")
pd.DataFrame(np.array(nonrutotal), index = nonrutime,
                   columns=[1,2,3,4,5,6,7,8,9,10])
                  

#### rumour topic by time

In [ ]:
rutotal = []
rutime = []
rumon = 1
for (nonrulda,rulda) in ldamodalcontainer:
    ru_ntopic = 1
    for idx, topic in rulda.print_topics(num_words=10):
        rutime.append("The {} mon, top {} topic".format(rumon,ru_ntopic))
        words = topic.split(" + ")
        words = [word.split("*") for word in words]
        words = [word[1] for word in words]
        rutotal.append(words)
        ru_ntopic += 1
        
    rumon = rumon + 1
print("=========== rumor topic by time ======================")
pd.DataFrame(np.array(rutotal), index = rutime,
                   columns=[1,2,3,4,5,6,7,8,9,10])

### User Analysis

In [ ]:
import json
import os
import re
import csv
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt

In [ ]:
index_with_empty_tweet = set()
train_raw = open("covid.data.txt", "r")
issue_index_train = 0
for event in train_raw:
    tweets = event.split(',')
    if (not os.path.isfile("./covid_data/" + tweets[0] + ".json")):
        index_with_empty_tweet.add(issue_index_train)
    issue_index_train += 1
  
label_file = pd.read_csv("covid_id_label.csv")

In [ ]:
def jsonloader(jsonfile):
    try: 
        f = open(jsonfile, 'r')
        data = json.load(f)
        tweetobj = json.loads(data)
        f.close()
        return tweetobj
    except IOError:
        return None

In [ ]:
import csv
import json

information_file = open("user_information.csv", "w")
writer = csv.writer(information_file, lineterminator='\n')
writer.writerow(["tweetID", "userID", "location", "description", "followers_count", "friends_count", "count_ratio",
"favourites_count", "is_verified", "time_diff", "label"])

with open("covid_id_label.csv", newline='') as labelfile:
    with open("covid_trimmed_data.txt", newline='') as eventsfile:
        labelreader = csv.reader(labelfile)
        eventsreader = csv.reader(eventsfile)
        next(labelreader)
        for (label,events) in zip(labelreader,eventsreader):
            #loading files
            #templist = []
            source = events.pop()
            sourcetweet = jsonloader("covid_data/" + source + ".json")
            # if there is no sourcetweet, continue
            if sourcetweet: 
                user_id = sourcetweet["user"]["id"]

                location = sourcetweet["user"]["location"]
                location = re.sub("[^a-zA-Z0-9]+", " ",location)

                description = sourcetweet["user"]["description"]
                description = re.sub("[^a-zA-Z0-9]+", " ",description)

                followers_count = sourcetweet["user"]["followers_count"]
                friends_count = sourcetweet["user"]["friends_count"]
                count_ratio = 0 if friends_count == 0 else followers_count/friends_count
                favourites_count = sourcetweet["user"]["favourites_count"]
                is_verified = 1 if sourcetweet["user"]["verified"] else 0

                tweet_creation_time = datetime.datetime.strptime(sourcetweet["created_at"], '%a %b %d %H:%M:%S %z %Y')
                account_creation_time = datetime.datetime.strptime(sourcetweet["user"]["created_at"], '%a %b %d %H:%M:%S %z %Y')
                time_diff = (tweet_creation_time - account_creation_time).days 

                cur_label = label[1]
                writer.writerow([source, user_id, location, description, followers_count, friends_count, count_ratio, favourites_count, is_verified, time_diff, cur_label])

            else: continue

In [ ]:
user_information = pd.read_csv("user_information.csv")

In [ ]:
import matplotlib.pyplot as plt
followers_count = user_information[["favourites_count"]][user_information["label"] == 0]
counts, bins = np.histogram(followers_count, bins=[800, 1000, 1500, 2000, 2500, 3000, 3500, 4000, 4500, 5000, 6000, 7000, 8000, 9000, 10000])

fig, ax = plt.subplots()
ax.bar(bins[:-1], counts, width=np.diff(bins), edgecolor="black", align="edge")
plt.xlabel("bins")
plt.ylabel("number")
plt.title("rumour user followers/friends")
plt.show()

In [ ]:
data = pd.DataFrame()
count = user_information["favourites_count"][user_information["label"] == 1]
data['bins'] = pd.cut(count,bins=[500,2000,5000,10000,100000], labels=["500-2000","2001-5000","5001-10000","10000+"])
data = data.groupby('bins').size()

In [ ]:
label = ["500-2000","2001-5000","5001-10000","10000+"]
title = plt.title('rumour user verified followers/friends')
plt.pie(data, labels=label, autopct='%1.1f%%', explode=(0, 0, 0.5, 0), shadow=True)

plt.axis('equal')
plt.show()

In [ ]:
time_diff = user_information[["time_diff"]][user_information["label"] == 1]
counts, bins = np.histogram(time_diff, bins=[1, 10, 30, 60, 182, 365])

time_diff2 = user_information[["time_diff"]][user_information["label"] == 0]
counts2, bins2 = np.histogram(time_diff2, bins=[1, 10, 30, 60, 182, 365])


fig, ax = plt.subplots()

bincenters = 0.5*(bins[1:]+bins[:-1])
plt.plot(bincenters,counts,label="rumour")

bincenters2 = 0.5*(bins2[1:]+bins2[:-1])
plt.plot(bincenters2,counts2,label="non-rumour")

plt.xlabel("time_diff range")
plt.ylabel("number")
plt.title("rumour user followers/friends")
plt.legend(loc='best')
plt.show()

In [ ]:
data = pd.DataFrame()
count = user_information["time_diff"][user_information["label"] == 1][user_information["is_verified"] == 0]
data['bins'] = pd.cut(count,bins=[1, 30, 60, 182, 365, 400])
data = data.groupby('bins').size()

In [ ]:
label = ["1-1month", "1-2months", "2-6months", "6months-1year", "1year+"]
title = plt.title('non-rumour user time difference')
plt.pie(data, labels=label, autopct='%1.1f%%', shadow=True)

plt.axis('equal')
plt.show()